In [ ]:
class WerewolfGame(Team):
    """Use the "software company paradigm" to hold a werewolf game"""
    env: Optional[WerewolfEnv] = None
    def __init__(self, context: Context = None, **data: Any):
        super(Team, self).__init__(**data)
        ctx = context or Context()
        if not self.env:
            self.env = WerewolfEnv(context=ctx)
        else:
            self.env.context = ctx  # The `env` object is allocated by deserialization

In [1]:
class WerewolfEnv(WerewolfExtEnv, Environment):
    round_cnt: int = Field(default=0)
class WerewolfExtEnv(ExtEnv):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    players_state: dict[str, tuple[str, RoleState]] = Field(
        default_factory=dict, description="the player's role type and state by player_name"
    )
    round_idx: int = Field(default=0)  # the current round
    step_idx: int = Field(default=0)  # the current step of current round
    eval_step_idx: list[int] = Field(default=[])
    per_round_steps: int = Field(default=len(STEP_INSTRUCTIONS))
    # game global states
    game_setup: str = Field(default="", description="game setup including role and its num")
    special_role_players: list[str] = Field(default=[])
    winner: Optional[str] = Field(default=None)
    win_reason: Optional[str] = Field(default=None)
    witch_poison_left: int = Field(default=1, description="should be 1 or 0")
    witch_antidote_left: int = Field(default=1, description="should be 1 or 0")
    # game current round states, a round is from closing your eyes to the next time you close your eyes
    round_hunts: dict[str, str] = Field(default_factory=dict, description="nighttime wolf hunt result")
    round_votes: dict[str, str] = Field(
        default_factory=dict, description="daytime all players vote result, key=voter, value=voted one"
    )
    player_hunted: Optional[str] = Field(default=None)
    player_protected: Optional[str] = Field(default=None)
    is_hunted_player_saved: bool = Field(default=False)
    player_poisoned: Optional[str] = Field(default=None)
    player_current_dead: list[str] = Field(default=[])

NameError: name 'WerewolfExtEnv' is not defined

In [2]:
##运行代码详情
#导入角色和游戏相关依赖
import asyncio
import fire

from metagpt.ext.werewolf.roles import Guard, Moderator, Seer, Villager, Werewolf, Witch#守卫 主持人 先知 村民 狼人 巫师
from metagpt.ext.werewolf.roles.human_player import prepare_human_player
from metagpt.ext.werewolf.werewolf_game import WerewolfGame
from metagpt.logs import logger

#由于MetaGPT是异步框架，使用asyncio启动游戏
async def start_game(
    investment: float = 20.0,
    n_round: int = 5,#回合数，建议n_round值设置小一点
    shuffle: bool = True,
    add_human: bool = False,
    use_reflection: bool = True,
    use_experience: bool = False,
    use_memory_selection: bool = False,
    new_experience_version: str = "",
):
    game = WerewolfGame()
    #初始化游戏设置
    game_setup, players = game.env.init_game_setup(
        role_uniq_objs=[Villager, Werewolf, Guard, Seer, Witch],#设置游戏玩家职业
        num_werewolf=2,
        num_villager=2,
        shuffle=shuffle,#是否打乱职业顺序，默认打乱
        add_human=add_human,#设置真人也参与游戏
        use_reflection=use_reflection,#是否让智能体对对局信息反思，默认开启
        use_experience=use_experience,#是否让智能体根据过去行为优化自身动作，默认关闭
        use_memory_selection=use_memory_selection,
        new_experience_version=new_experience_version,
        prepare_human_player=prepare_human_player,
    )
    logger.info(f"{game_setup}")

    players = [Moderator()] + players#主持人加入游戏
    game.hire(players)
    game.invest(investment)
    game.run_project(game_setup)#主持人广播游戏情况
    await game.run(n_round=n_round)


def main(
    investment: float = 20.0,
    n_round: int = 5,#运行前建议将此处n_round修改小一点，否则对钱包不友好！！！
    shuffle: bool = True,
    add_human: bool = False,
    use_reflection: bool = True,
    use_experience: bool = False,
    use_memory_selection: bool = False,
    new_experience_version: str = "",
):
    asyncio.run(
        start_game(
            investment,
            n_round,
            shuffle,
            add_human,
            use_reflection,
            use_experience,
            use_memory_selection,
            new_experience_version,
        )
    )


if __name__ == "__main__":
    fire.Fire(main)

ModuleNotFoundError: No module named 'metagpt.ext'